In [1]:
pip install git+https://github.com/JustAnotherArchivist/snscrape.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-4okyjmcc
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-4okyjmcc
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 614d4c2029a62d348ca56598f87c425966aaec66
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for snscrape: filename=snscrape-0.7.0.20230622-py3-none-any.whl size=74814 sha256=83a6bfbbf1d7f6b7412c71b6379375bc0cfa15df5456609bcf5263a44864fe00
  Stored in directory: /tmp/pip-ephem-wheel-cache-mz8u2m56/wheels/05/e9/f7/57056e7c7e44b1feed932fa49fdec9d706c4f563e37160ab74
Successfully built snscrape


In [2]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime, timedelta

In [3]:
# Hashtag
hashtag = "#OceanGate"

In [16]:
# Tarih aralığı
today = datetime.now().date()
month_ago = today - timedelta(days=15)

In [17]:
# Sorgu
query = f'{hashtag} since:{month_ago}'

In [19]:
tweets_list = []
# max çekilecek tweet sayısı
max_tweets = 1000
for i, tweet in enumerate(sntwitter.TwitterHashtagScraper(query).get_items()):
    if i >= max_tweets:
        break
    tweets_list.append([tweet.date, tweet.id,tweet.username,tweet.user.id,tweet.hashtags, tweet.content, tweet.likeCount, tweet.retweetCount, tweet.replyCount])

<ipython-input-19-899cfd7cfc40>:7: DeprecatedFeatureWarning: username is deprecated, use user.username instead
  tweets_list.append([tweet.date, tweet.id,tweet.username,tweet.user.id,tweet.hashtags, tweet.content, tweet.likeCount, tweet.retweetCount, tweet.replyCount])
<ipython-input-19-899cfd7cfc40>:7: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets_list.append([tweet.date, tweet.id,tweet.username,tweet.user.id,tweet.hashtags, tweet.content, tweet.likeCount, tweet.retweetCount, tweet.replyCount])


In [20]:
# Verileri DataFrame'e dönüştürme
df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'username' ,'userid','hastags' ,  'Content', 'Like Count', 'Retweet Count', 'Reply Count'])

In [21]:
df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_localize(None)

In [22]:
df.to_excel('oceangate.xlsx', index=False)

In [23]:
# Verileri Excel dosyasından okuma
df = pd.read_excel('oceangate.xlsx')

In [24]:
# Düzenlenmiş verileri tutacak listeleri oluşturma
source_list = []
target_list = []
type_list = []
weight_list = []

In [25]:
# Veri setini döngüye alarak düzenleme işlemlerini gerçekleştirme
for index, row in df.iterrows():
    hashtags = row['hastags']
    username = row['username']
    like_count = row['Like Count']
    retweet_count = row['Retweet Count']
    reply_count = row['Reply Count']


    # Hashtag'leri ayrıştırma
    hashtag_list = hashtags.strip("[]").replace("'", "").split(',')

    # Hashtag listesini döngüye alarak düğüm ve kenarları oluşturma
    for hashtag in hashtag_list:
        hashtag = hashtag.strip()  # Boşlukları kaldırma

        # hastag ismi
        # birden çok hastag varsa alt alta o hastagleri de yaz
        if hashtag.lower() == 'oceangate':
            source = 'oceangate'
            target = username
            relation_type = 'mention'
            weight = 1+like_count + retweet_count + reply_count
            source_list.append(source)
            target_list.append(target)
            type_list.append(relation_type)
            weight_list.append(weight)

In [26]:
# DataFrame'i oluştur
data = {
    'Source': source_list,
    'Target': target_list,
    'Type': type_list,
    'Weight': weight_list
}

In [27]:
df_new = pd.DataFrame(data)

In [28]:
df_new.to_csv('oceangate_gephi', index=False)